<a href="https://colab.research.google.com/github/tjdrhks0808/Project-Road_Accident/blob/main/Daejeon_Externel_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 카카오 API 활용한 변수 추출

In [ ]:
from geoband.API import *
from geopandas.tools import sjoin


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString

import pydeck as pdk
import shapefile
import os
import re
import matplotlib 
matplotlib.font_manager._rebuild()
import seaborn as sns

import mapboxgl
from mapboxgl.viz import *
from mapboxgl.utils import create_color_stops

from matplotlib import font_manager, rc
import platform

import requests; from urllib.parse import urlparse

In [ ]:
GetCompasData('SBJ_2102_003', '2', '2.대전광역시_교통사고격자(2017~2019).geojson')
acc_cnt = gpd.read_file('2.대전광역시_교통사고격자(2017~2019).geojson')

## 주요시설물(카카오 api) 수집

> 카카오api를 활용하여 외부데이터를 수집한다 

&nbsp;

1. 중심좌표 구하기

In [ ]:
acc_cnt_pt = acc_cnt.copy() # copy GeoDataFrame
acc_cnt_pt['geometry'] = acc_cnt_pt['geometry'].centroid  # centroid 공간함수로 중심좌표 반환 

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


&nbsp;

2. point값을 x,y 좌표계로 변환

In [ ]:
def getXY(pt):       #변환함수
    return (pt.x, pt.y)

x,y = [list(t) for t in zip(*map(getXY, acc_cnt_pt['geometry']))]

In [ ]:
acc_cnt_pt['x']=x
acc_cnt_pt['y']=y

&nbsp;

3. 카카오톡 API 활용

  - 좌표, 반경, 시설물의 keyword, 카테고리, 승인키를 입력하면 중심좌표에서 입력 반경 내에 있는 시설물의 수를 알 수 있도록 하는 함수 정의

In [ ]:
def point_to_num_cat(y,x,radius,keyword,category, key):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?&y=" + y + "&x=" + x + "&radius=" + radius + "&query=" + keyword  +"&category_group_code" + category
    result = requests.get(urlparse(url).geturl(),
                      headers={"Authorization":key})
    json_obj = result.json()
    global val
    val = json_obj['meta']['pageable_count']
    return val


# key list
# "KakaoAK d3c61988f98633f8311396ae5ed20fa2"
# "KakaoAK cb77b05f141e13a1f58eb815c768e57e"
# "KakaoAK df957471473cace65fa87f01433819de"
# "KakaoAK 0973431c81a22279eb5f396f1ba570d2"
# "KakaoAK 21c25d923a619a70cba3e38b7d594472"

### 반경 50M 내 지하철역의 수

In [ ]:
ls_subway = []                                                
for i in range(0,len(acc_cnt_pt['gid'])): 
    ls_subway.append(point_to_num_cat(str(acc_cnt_pt['y'][i]), str(acc_cnt_pt['x'][i]), '50', '역','SW8',
                                     "KakaoAK d3c61988f98633f8311396ae5ed20fa2"))

In [ ]:
acc_cnt_pt['subway_cnt'] = ls_subway
subway_cnt = acc_cnt_pt.loc[:, ['gid', 'subway_cnt']]

subway_cnt.to_file('subway_cnt.geojson', driver='GeoJSON')

### 반경 50M 내 주차장의 수

In [ ]:
ls_parking = []                                                
for i in range(0,len(acc_cnt_pt['gid'])): 
    ls_parking.append(point_to_num_cat(str(acc_cnt_pt['y'][i]), str(acc_cnt_pt['x'][i]), '50', '주차','PK6',
                                            "KakaoAK cb77b05f141e13a1f58eb815c768e57e"))

In [ ]:
acc_cnt_pt['parkinglot_cnt'] = ls_parking
parkinglot_cnt = acc_cnt_pt.loc[:, ['gid', 'parkinglot_cnt']]

parkinglot_cnt.to_file('parkinglot_cnt.geojson', driver='GeoJSON')

### 반경 50M 내 학교의수

In [ ]:
ls_scl = []                                                
for i in range(0,len(acc_cnt_pt['gid'])): 
    ls_scl.append(point_to_num_cat(str(acc_cnt_pt['y'][i]), str(acc_cnt_pt['x'][i]), '50', '학교','SC4',
                                         "KakaoAK df957471473cace65fa87f01433819de"))

In [ ]:
acc_cnt_pt['school_cnt'] = ls_scl
school_cnt = acc_cnt_pt.loc[:, ['gid', 'school_cnt']]

school_cnt.to_file('school_cnt.geojson', driver='GeoJSON')

### 대형마트

In [ ]:
ls_mart = []                                                
for i in range(0,len(acc_cnt_pt['gid'])): 
    ls_mart.append(point_to_num_cat(str(acc_cnt_pt['y'][i]), str(acc_cnt_pt['x'][i]), '50', '마트','MT1',
                                   "KakaoAK 0973431c81a22279eb5f396f1ba570d2"))

In [ ]:
acc_cnt_pt['ssm_cnt'] = ls_mart
ssm_cnt = acc_cnt_pt.loc[:, ['gid', 'ssm_cnt']]

ssm_cnt.to_file('ssm_cnt.geojson', driver='GeoJSON')